In [162]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

In [163]:
df_cleaned = pd.read_csv('../data/train_cleaned.csv')
df_dirty_test = pd.read_csv('../data/test_cleaned.csv')
df_cleaned.head()

,Unnamed: 0,title,address,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,price,built_year,tenure,available_unit_types
0,0,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,3.0,2.0,1115,unspecified,1.414399,103.837196,yishun south,yishun,514500.0,1988.0,99-year leasehold,NaN
1,1,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,4.0,2.0,1575,unspecified,1.372597,103.875625,serangoon north,serangoon,995400.0,1992.0,99-year leasehold,"1, 2, 3, 4, 5, 6 br"
2,2,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,4.0,6.0,3070,partial,1.298773,103.895798,mountbatten,marine parade,8485000.0,2022.0,freehold,"studio, 3, 4, 5, 6 br"
3,3,3 bed condo for sale in leedon green,26 leedon heights,leedon green,condo,3.0,2.0,958,partial,1.312364,103.803271,farrer court,bukit timah,2626000.0,2023.0,freehold,"studio, 1, 2, 3, 4 br"
4,4,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,2.0,1.0,732,unspecified,1.273959,103.843635,anson,downtown core,1764000.0,2026.0,99-year leasehold,"studio, 1, 2, 3, 4, 5 br"


Singapore has the following latitude and longitude coordinates in its extreme ends:
1. left-most (Tuas) :  1.30871,103.64287
2. right-most (Changi) : 1.34538,104.00270
3. top-most (Sembawang) : 1.46227,103.79487
4. bottom-most (Bukit Merah) : 1.28762,103.82467


#### Min latitude - 1.28762       Max latitude - 1.46227

#### Min longitude - 103.64         Max longitude - 104.00

But we can see that in the data, min longitude is -77.065364 and max latitude is 69.486768 which are out of the range of latitude and longitude values 

<img src="images/singapore-lat-long-map.jpeg" width=600 height=600 />



In [164]:
df_cleaned['built_year'] = df_cleaned['built_year'].astype(int)
df_cleaned['num_beds'] = df_cleaned['num_beds'].astype(int)
df_cleaned['num_baths'] = df_cleaned['num_baths'].astype(int)
df_cleaned['num_beds'] = df_cleaned['num_beds'].astype(int)
df_cleaned['num_baths'] = df_cleaned['num_baths'].astype(int)
df_cleaned['lng'] = df_cleaned['lng'].astype(np.float16)
df_cleaned['lat'] = df_cleaned['lat'].astype(np.float16)
df_cleaned.info()

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [165]:
#print(df_cleaned.lng.max)
df_max_lng = df_cleaned[df_cleaned.lng > 121.0]
df_min_lng = df_cleaned[df_cleaned.lng < -77.0]
df_max_lat = df_cleaned[df_cleaned.lat > 69.0]
df_wrong_coordinates = pd.concat([df_max_lng, df_min_lng, df_max_lat])

df_wrong_coordinates

# it is interesting to note that in all the records where latitude and longitude have incorrect coordinates,
# "planning_area" and "subzone" have missing values, this can also be verified by checking for count of missing values

,Unnamed: 0,title,address,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,price,built_year,tenure,available_unit_types
28,28,3 bed condo for sale in 1953,1 tessensohn road,1953,apartment,3.0,3.0,1130,unspecified,14.484814,121.023232,NaN,NaN,2671200.0,2023.0,freehold,"studio, 1, 2, 3, 4, 5, 6 br"
56,56,4 bed condo for sale in 1953,1 tessensohn road,1953,apartment,4.0,4.0,1399,unspecified,14.484814,121.023232,NaN,NaN,3272800.0,2023.0,freehold,"studio, 1, 2, 3, 4, 5, 6 br"
216,216,3 bed condo for sale in 1953,1 tessensohn road,1953,condo,3.0,3.0,1152,unspecified,14.484814,121.023232,NaN,NaN,2292200.0,2023.0,freehold,"studio, 1, 2, 3, 4, 5, 6 br"
480,480,2 bed condo for sale in 1953,1 tessensohn road,1953,condo,2.0,2.0,1130,unfurnished,14.484814,121.023232,NaN,NaN,2363600.0,2023.0,freehold,"studio, 1, 2, 3, 4, 5, 6 br"
580,580,3 bed condo for sale in 1953,1 tessensohn road,1953,apartment,3.0,3.0,1119,unspecified,14.484814,121.023232,NaN,NaN,2163000.0,2023.0,freehold,"studio, 1, 2, 3, 4, 5, 6 br"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,82,1 bed condo for sale in pollen & bleu,15 farrer drive,pollen & bleu,condo,1.0,1.0,549,unspecified,69.486768,20.184434,NaN,NaN,1470000.0,2017.0,99-year leasehold,"1, 2, 3, 4, 5 br"
2675,2675,2 bed condo for sale in pollen & bleu,17 farrer drive,pollen & bleu,condo,2.0,2.0,1163,fully,69.486768,20.184434,NaN,NaN,2362500.0,2017.0,99-year leasehold,"1, 2, 3, 4, 5 br"
8520,8520,3 bed condo for sale in pollen & bleu,17 farrer drive,pollen & bleu,condo,3.0,2.0,1184,unfurnished,69.486768,20.184434,NaN,NaN,2467500.0,2017.0,99-year leasehold,"1, 2, 3, 4, 5 br"
11103,11103,3 bed condo for sale in pollen & bleu,15 farrer drive,pollen & bleu,condo,3.0,2.0,1163,unspecified,69.486768,20.184434,NaN,NaN,2058000.0,2017.0,99-year leasehold,"1, 2, 3, 4, 5 br"


In [166]:
print(df_wrong_coordinates.shape)
print(df_cleaned["subzone"].isnull().sum())
print(df_cleaned["planning_area"].isnull().sum())

(106, 17)
106
106


In [167]:
# coordinates are incorrect for 5 'address'
df_wrong_coordinates["address"].value_counts()

1 tessensohn road       90
38 lorong 32 geylang     6
5 jalan mutiara          5
17 farrer drive          3
15 farrer drive          2
Name: address, dtype: int64

In [168]:
# using the 'address' we can manually correct the latitude, longitude coordinates along with 
# filling of values for sub zone and planning_area

df_cleaned.loc[df_cleaned.address == "1 tessensohn road", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', '1.3164313', '103.8575321', 'balestier', 'novena'
df_cleaned.loc[df_cleaned.address == "38 lorong 32 geylang", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', '1.31262', '103.88686', 'aljunied', 'geylang'
df_cleaned.loc[df_cleaned.address == "5 jalan mutiara", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', '1.29565', '103.82887', 'leonie hill', 'river valley'
df_cleaned.loc[df_cleaned.address == "17 farrer drive", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', '1.313259', '103.806622', 'holland road', 'bukit timah'
df_cleaned.loc[df_cleaned.address == "15 farrer drive", 
               ['property_type', 'lat', 'lng', 'subzone', 'planning_area']] = 'condo', '1.313259', '103.806622', 'holland road', 'bukit timah'


In [169]:
print(df_cleaned['property_type'].value_counts())
# changing "hdb 3 rooms", "hdb 4 rooms" and likewise to "hdb" since the number of rooms info can
# be obtained from "num_beds"
df_cleaned['property_type'].mask(df_cleaned['property_type'].str.contains("hdb"), "hdb", inplace=True)
df_cleaned.drop(df_cleaned[df_cleaned['property_type'] == 'land only'].index, inplace = True)
print(df_cleaned['property_type'].value_counts())

condo                  9248
hdb                    2783
hdb 3 rooms            1189
hdb 4 rooms            1118
semi-detached house    1007
executive condo         918
bungalow                740
hdb 5 rooms             677
apartment               482
terraced house          441
hdb executive           302
corner terrace          253
hdb 2 rooms             234
cluster house           183
conservation house       21
landed                   19
walk-up                  14
townhouse                 9
good class bungalow       5
land only                 2
shophouse                 1
Name: property_type, dtype: int64
condo                  9248
hdb                    6303
semi-detached house    1007
executive condo         918
bungalow                740
apartment               482
terraced house          441
corner terrace          253
cluster house           183
conservation house       21
landed                   19
walk-up                  14
townhouse                 9
good class bun

In [170]:
print(df_cleaned['furnishing'].value_counts())
# dropping 7 records with "na" 
df_cleaned = df_cleaned[df_cleaned.furnishing != "na"]

unspecified    14338
partial         2969
unfurnished     1811
fully            519
na                 7
Name: furnishing, dtype: int64


In [171]:
print("before changes")
print("condo")
print(df_cleaned["title"].str.contains("condo").value_counts())
print("\n")
print("apartment")
print(df_cleaned["title"].str.contains("apartment").value_counts())
print("\n")
print("property_type")
print(df_cleaned["property_type"].value_counts())


#The count of condos and apartments in title and property_type does not match

before changes
condo
False    9897
True     9740
Name: title, dtype: int64


apartment
False    19631
True         6
Name: title, dtype: int64


property_type
condo                  9246
hdb                    6303
semi-detached house    1007
executive condo         918
bungalow                740
apartment               477
terraced house          441
corner terrace          253
cluster house           183
conservation house       21
landed                   19
walk-up                  14
townhouse                 9
good class bungalow       5
shophouse                 1
Name: property_type, dtype: int64


In [172]:
df_cleaned.loc[(df_cleaned['property_type']!="condo") & 
       (df_cleaned['title'].str.contains('condo')),['property_type']] = "condo"
df_cleaned.loc[(df_cleaned['property_type']!="apartment") & 
       (df_cleaned['title'].str.contains('apartment')),['property_type']] = "apartment"

In [173]:
print("after changes")
print("condo")
print(df_cleaned["title"].str.contains("condo").value_counts())
print("\n")
print("apartment")
print(df_cleaned["title"].str.contains("apartment").value_counts())
print("\n")
print("property_type")
print(df_cleaned["property_type"].value_counts())


after changes
condo
False    9897
True     9740
Name: title, dtype: int64


apartment
False    19631
True         6
Name: title, dtype: int64


property_type
condo                  9734
hdb                    6303
semi-detached house    1006
executive condo         918
bungalow                740
terraced house          440
corner terrace          252
cluster house           183
conservation house       21
landed                   19
townhouse                 9
apartment                 6
good class bungalow       5
shophouse                 1
Name: property_type, dtype: int64


In [174]:
#g.	Language translation

In [175]:
#h.	Handle missing values

df_cleaned.isnull().sum()

Unnamed: 0                 0
title                      0
address                    0
property_name              0
property_type              0
num_beds                   0
num_baths                  0
size_sqft                  0
furnishing                 0
lat                        0
lng                        0
subzone                    0
planning_area              0
price                      0
built_year               900
tenure                   198
available_unit_types    1430
dtype: int64

In [176]:
#i. Features

#1. title - DROPPING
#2. address - DROPPING
#3. property_name - KEEP
#4. property_type - ordinal encoding
#5. tenure - one-hot encoding (reduce to 3 columns - freehold, 99-year, 999-year)
#6. built_year - handle null values
#6.1 numb_beds/num_baths/size_sqft - KEEP AS IT IS
#7. floor_level - ordinal encoding (DROPPING)
#8. furnishing - one-hot encoding
#9. available_unit_types -  (ON HOLD/DROPPING)
#10. total_num_units - DROPPING
#11. lat/lng - KEEP AS IT IS
#12. planning_area - one-hot encoding/adarsh to work on map
#13. subzone - one-hot encoding/Adarsh to work on map
#14. ADD FEATURES - number os schools/MRT stations/malls within a pre-fixed radius

In [177]:
#No need to run this unless all data has been cleaned.
#df_dirty_test.to_csv('../data/test_cleaned.csv')  
df_cleaned.to_csv('../data/train_cleaned.csv')